In [18]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [20]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].fillna("missing")

In [21]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
# data.dropna(axis=0, thresh=int(data.shape[1]*0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0]*0.5), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)

In [22]:
data

,КАН,АКР,Гормональная активность 0-нет 1-да,"Максимальный размер, мм","Нативная плотность, НU",APW % КВ - коэф вымывания,Weiss,Кортизол крови утро,Кортизол крови вечер,Кортизол после пробы с 1 мг дексаметазона (ДМТ),...,THE_P3,THB_THA,THF_THE1,THF_THE,aTHF_THF,aTHB_THB,11An_11Et,THE_THS,THE_11oxoP3,16-OHdP2-3a/16-OHdP2-3b
1,0.0,1.0,1.0,181.0,NaN,NaN,7.0,NaN,NaN,NaN,...,0.722438,0.268293,0.837273,0.276251,2.030846,26.136364,2.827039,0.312336,334.200000,1.205818
2,0.0,1.0,1.0,137.0,NaN,NaN,6.0,681.0,739.0,NaN,...,17.868078,5.948276,1.601304,1.245525,0.285646,0.289855,0.670094,8.016807,353.903226,0.795526
3,0.0,1.0,1.0,79.0,48.0,NaN,4.0,1418.0,1021.0,NaN,...,3.100000,1.000000,0.950000,0.770000,0.230000,0.170000,1.780000,16.680000,25.010000,2.530000
4,0.0,1.0,1.0,54.0,18.0,53.330000,3.0,687.0,628.0,NaN,...,5.308889,10.000000,0.863495,0.692278,0.247324,2.400000,5.544041,27.941520,227.523810,2.403478
5,0.0,1.0,1.0,NaN,NaN,64.583333,NaN,NaN,NaN,NaN,...,0.558894,NaN,0.964336,0.883217,0.091845,0.465753,1.983240,1.279727,85.121212,1.422500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1.0,0.0,0.0,57.0,16.0,48.570000,2.0,304.0,113.0,39.0,...,4.273333,0.959184,0.413451,0.198456,1.083333,1.872340,2.837771,36.214689,NaN,1.371429
161,1.0,0.0,1.0,36.0,NaN,96.000000,2.0,344.0,328.0,344.0,...,1.991166,3.255814,0.648263,0.365631,0.773000,2.371429,1.818519,5.053812,13.828221,NaN
162,1.0,0.0,1.0,17.0,8.0,96.428571,0.0,784.0,245.0,NaN,...,37.064646,1.168342,0.929232,0.514090,0.807527,1.247312,3.576531,NaN,1528.916667,NaN
163,1.0,0.0,1.0,43.0,32.0,50.000000,9.0,628.0,345.0,60.4,...,3.629134,4.057143,0.628047,0.307312,1.043678,1.577465,5.618234,219.476190,307.266667,3.089655


In [23]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=314, shuffle=True)

In [45]:
from catboost import CatBoostClassifier
import optuna

def objective(trial):
    param = {
    'verbose':0,
    'thread_count':-1,
    'cat_features':["Гормональная активность 0-нет               1-да"],
    'loss_function': "Logloss",
    'iterations': trial.suggest_int('iterations', 10, 100),
    'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
    'depth': trial.suggest_int('depth', 3, 10),
    'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
    'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
    'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 0, 8)
    }

    model = CatBoostClassifier(**param, task_type='GPU', random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-12-24 16:25:59,355] A new study created in memory with name: no-name-aa9385dc-59bb-4d27-a0d9-651ba849807c
[I 2024-12-24 16:26:02,928] Trial 0 finished with value: 1.0 and parameters: {'iterations': 23, 'learning_rate': 0.015061570992288376, 'depth': 10, 'l2_leaf_reg': 8, 'boosting_type': 'Ordered', 'max_ctr_complexity': 5}. Best is trial 0 with value: 1.0.
[I 2024-12-24 16:26:05,040] Trial 1 finished with value: 1.0 and parameters: {'iterations': 81, 'learning_rate': 0.021050186244197005, 'depth': 3, 'l2_leaf_reg': 10, 'boosting_type': 'Ordered', 'max_ctr_complexity': 3}. Best is trial 0 with value: 1.0.
[I 2024-12-24 16:26:08,568] Trial 2 finished with value: 0.9411764705882353 and parameters: {'iterations': 64, 'learning_rate': 0.02909737827554296, 'depth': 8, 'l2_leaf_reg': 2, 'boosting_type': 'Ordered', 'max_ctr_complexity': 5}. Best is trial 0 with value: 1.0.
[I 2024-12-24 16:26:10,539] Trial 3 finished with value: 1.0 and parameters: {'iterations': 53, 'learning_rate': 0

KeyboardInterrupt: 

In [55]:
model = CatBoostClassifier(**study.best_params, random_state=63)

In [58]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(shuffle=True, n_splits=10, random_state=42)

cv = cross_validate(model, X, y, n_jobs=-1,
                    scoring=["accuracy",
                             "f1",
                             "precision",
                             "recall"],
                    cv=skf,
                    return_estimator=True)

In [60]:
cv["test_accuracy"].mean()

0.9143382352941177

In [61]:
for i in range(5):
    print(
        " accuracy", cv["test_accuracy"][i], "\n",
        "f1", cv["test_f1"][i], "\n",
        "recall", cv["test_recall"][i], "\n",
        "precision", cv["test_precision"][i], "\n"
    )
    print("---------------------------------------------")

 accuracy 1.0 
 f1 1.0 
 recall 1.0 
 precision 1.0 

---------------------------------------------
 accuracy 0.8823529411764706 
 f1 0.875 
 recall 0.875 
 precision 0.875 

---------------------------------------------
 accuracy 0.8823529411764706 
 f1 0.8571428571428571 
 recall 0.75 
 precision 1.0 

---------------------------------------------
 accuracy 0.9411764705882353 
 f1 0.9333333333333333 
 recall 0.875 
 precision 1.0 

---------------------------------------------
 accuracy 0.875 
 f1 0.875 
 recall 0.875 
 precision 0.875 

---------------------------------------------


In [62]:
feature_importances = model.get_feature_importance()
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

importance_df

CatBoostError: Model has no meta information needed to calculate feature importances.                             Pass training dataset to this function.